In [ ]:
import sqlite3
import pandas as pd

# 1. Load your data
df = pd.read_csv('cleaned_retail_data.csv')

# 2. Create (or connect to) a local database file
# This will create a file named 'RetailProject.db' in my folder
conn = sqlite3.connect('RetailProject.db')

# 3. Upload the data
df.to_sql('cleaned_retail', conn, if_exists='replace', index=False)

print("✅ Data successfully uploaded to SQLite!")

# 4. Verify with a quick SQL query
query = "SELECT * FROM cleaned_retail LIMIT 5"
check_df = pd.read_sql(query, conn)
print(check_df)

# Close the connection
conn.close()

In [ ]:
import sqlite3
import pandas as pd

# 1. Connect to your SQLite database
conn = sqlite3.connect('RetailProject.db')

# 2. SQL Query to create the Customer-Level Table
# We use '2011-12-10' as our reference point (the day after the last sale)
rfm_query = """
SELECT 
    CustomerID,
    MAX(Country) AS Country,
    COUNT(DISTINCT order_ID) AS Frequency,
    SUM(TotalAmount) AS Monetary,
    CAST(julianday('2011-12-10') - julianday(MAX(Purchase_Date)) AS INT) AS Recency
FROM cleaned_retail
GROUP BY CustomerID
"""

# 3. Load the results into a DataFrame
df_customer_stats = pd.read_sql(rfm_query, conn)

# 4. Define Churn (Target Variable)
# 90 days is a standard retail benchmark for 'churn'
df_customer_stats['Is_Churned'] = df_customer_stats['Recency'].apply(lambda x: 1 if x > 90 else 0)

# 5. Save this for Power BI
df_customer_stats.to_csv('Customer_Churn_Master.csv', index=False)

print("✅ Customer-level RFM table created!")
print(df_customer_stats.head())

conn.close()